# Base NN Architectures

In [1]:
import torch
from torch import nn
from scaling.models.model_factory import MODELS
from scaling.train import train

[rank: 0] Seed set to 42


In [2]:
def count_params(model):
    return sum([x.numel() for x in model.parameters() if x.requires_grad])

@torch.no_grad()
def check_model(model, channels=12, seq_len=1024):
    """Runs one forward on random data."""
    _ = model(torch.rand(1, channels, seq_len))

In [3]:
print("\033[4mMODEL NAME\tPARAMETERS\033[0m")
for model_name, model_fn in MODELS.items():
    model = model_fn()
    params = count_params(model)
    print(f"{model_name}\t{params:,}")
    check_model(model)

MODEL NAME	PARAMETERS
convnext_mini	13,383,098
convnext_tiny	26,787,770
convnext_small	48,132,026
convnext_base	85,458,842
convnext_large	192,036,698
resnet18	3,862,170
resnet50	16,012,442
resnet101	28,319,898
resnext18	12,867,482
resnext50	22,086,042
resnext101	79,676,826
vit_tiny	6,520,906
vit_small	25,638,490
vit_base	85,725,866
vit_large	303,376,282
getemed_small	952,770
getemed_base	3,160,770
getemed_large	13,558,210


# Dry run all models

In [3]:
path = "/sc-scratch/sc-scratch-gbm-radiomics/ecg/physionet_challenge/training_pt/metadata_v4.csv"
loss_fn = nn.BCEWithLogitsLoss()

In [4]:
for key in MODELS.keys():
    train(
        project="test",
        name="test_run_1",
        meta_file_path=path,
        fold=0,
        model_name=key,
        loss_fn=loss_fn,
        lr_decay_gamma=0.95,
        fast_dev_run=True,
    )

/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
You are using a CUDA device ('NVIDIA A100-PCIE-40GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name   

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  2.44it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]                 

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  1.98it/s]


/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type              | Params | Mode 
------------------------------------------------------
0 | model   | ConvNeXt          | 26.8 M | train
1 | loss_fn | BCEWithLogitsLoss | 0      | train
------------------------------------------------------
26.8 M    Trainable params
0         Non-trainable params
26.8 M    Total params
107.151   Total estimated model par

...

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.35it/s]


/home/jabareen/.conda/envs/ecg/lib/python3.12/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /home/jabareen/.conda/envs/ecg/lib/python3.12/site-p ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name    | Type                 | Params | Mode 
---------------------------------------------------------
0 | model   | ECGVisionTransformer | 13.6 M | train
1 | loss_fn | BCEWithLogitsLoss    | 0      | train
---------------------------------------------------------
13.6 M    Trainable params
0         Non-trainable params
13.6 M    Total params
54.233    Total esti

Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  5.05it/s]
Validation: |          | 0/? [00:00<?, ?it/s]
Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.28it/s]                

`Trainer.fit` stopped: `max_steps=1` reached.


Epoch 0: 100%|██████████| 1/1 [00:00<00:00,  3.27it/s]
